<a href="https://colab.research.google.com/github/Jafima7/EDEM2022/blob/main/Exercises_01_DFBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataFrames Basics Exercises

## Prerrequisites

Install Spark and Java in VM

In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz

In [2]:
ls -l # check the .tgz is there

total 267684
drwxr-xr-x 1 root root      4096 Dec  8 14:36 sample_data/
-rw-r--r-- 1 root root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz


In [3]:
# unzip it
!tar xf spark-3.3.1-bin-hadoop2.tgz

In [4]:
!pip install -q findspark

Defining the environment

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [6]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("DataFramesBasics Exercises") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.3.1'

In [7]:
spark

In [8]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [9]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [10]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/movies.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/cars.json -P /dataset

In [11]:
ls -l /dataset

total 1324
-rw-r--r-- 1 root root   74910 Dec 10 08:51 cars.json
-rw-r--r-- 1 root root 1274347 Dec 10 08:51 movies.json


## DataFrames Basics Exercises

1) Create a manual DF describing smartphones
  - maker
  - model
  - screen dimension
  - camera megapixels
  
2) Read another file from the dataset/ folder, e.g. movies.json
  - print its schema
  - count the number of rows, call count()

Exercise 1)

In [21]:
smartphones = spark.sparkContext.parallelize([
    ("Apple", "iPhone 12", 6, 10),
    ("Samsung", "A51", 8, 8),
    ("Samsung", "Galaxy Z Flip 4", 6, 10)])

In [22]:
smartphonesDF = smartphones.toDF(["maker", "model", "screen dimension", "camera megapixels"])

In [23]:
smartphonesDF.show()

+-------+---------------+----------------+-----------------+
|  maker|          model|screen dimension|camera megapixels|
+-------+---------------+----------------+-----------------+
|  Apple|      iPhone 12|               6|               10|
|Samsung|            A51|               8|                8|
|Samsung|Galaxy Z Flip 4|               6|               10|
+-------+---------------+----------------+-----------------+



Exercise 2)

In [25]:
moviesDF = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .load("/dataset/movies.json")

In [26]:
moviesDF.printSchema()
print(f"The Movies DF has {moviesDF.count()} rows")

root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)

The Movies DF has 3201 rows


## Columns and Expressions Exercises

1. Read the movies DF and select 2 columns of your choice
2. Create another column summing up the total profit of the movies = US_Gross + Worldwide_Gross + DVD sales. Are you pbtaining nulls? How you can solve it?
3. Select all COMEDY movies with IMDB rating above 6

Use as many versions as possible

Exercise 1)

In [29]:
moviesDF.select("Title", "Rotten_Tomatoes_Rating").show()

+--------------------+----------------------+
|               Title|Rotten_Tomatoes_Rating|
+--------------------+----------------------+
|      The Land Girls|                  null|
|First Love, Last ...|                  null|
|I Married a Stran...|                  null|
|Let's Talk About Sex|                    13|
|                Slam|                    62|
| Mississippi Mermaid|                  null|
|           Following|                  null|
|             Foolish|                  null|
|             Pirates|                    25|
|     Duel in the Sun|                    86|
|           Tom Jones|                    81|
|             Oliver!|                    84|
|To Kill A Mocking...|                    97|
|    Tora, Tora, Tora|                  null|
|   Hollywood Shuffle|                    87|
|Over the Hill to ...|                  null|
|              Wilson|                  null|
|        Darling Lili|                  null|
|The Ten Commandments|            

Exercise 2)

In [32]:
moviesDF.select("Title", col("US_Gross") + col("Worldwide_Gross")).show()

+--------------------+----------------------------+
|               Title|(US_Gross + Worldwide_Gross)|
+--------------------+----------------------------+
|      The Land Girls|                      292166|
|First Love, Last ...|                       21752|
|I Married a Stran...|                      406268|
|Let's Talk About Sex|                      747230|
|                Slam|                     2097340|
| Mississippi Mermaid|                     2649102|
|           Following|                       89410|
|             Foolish|                    12053816|
|             Pirates|                     7983650|
|     Duel in the Sun|                    40800000|
|           Tom Jones|                    75200000|
|             Oliver!|                    74805754|
|To Kill A Mocking...|                    26259692|
|    Tora, Tora, Tora|                    59096582|
|   Hollywood Shuffle|                    10457234|
|Over the Hill to ...|                     6000000|
|           

Exercise 3)

In [40]:
moviesDF.filter((moviesDF.Major_Genre == "Comedy") & (moviesDF.IMDB_Rating > 6)).show()

+--------------------+-----------------+--------------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|       Creative_Type|         Director|         Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|              Source|               Title|US_DVD_Sales| US_Gross|Worldwide_Gross|
+--------------------+-----------------+--------------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|                null|             null|           Lionsgate|        6.8|       865|       null|     Comedy|           250000|   28-Aug-98|                  null|            null|                null|I Married a Stran